In [1]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, IntProgress, Label
import os
import logging

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(), 'lib'))

In [3]:
from project_description import project_description
from settings import settings
from fragmax_logo import page_header
from log import OutputWidgetHandler
from db_objects import db_objects
from crystal_screen import crystal_screen
from crystal_plate import crystal_plate
from inspect_plate import inspect_plate
from soak_plate import soak_plate
from crystal_soak import crystal_soak
from mounted_crystals import mounted_crystals

In [4]:
# qgrid widget is responsible for voila problem
# see https://github.com/voila-dashboards/voila/issues/72
# the solution is to edit /Applications/anaconda3/share/jupyter/nbextensions/qgrid/index.js and change
# define(["@jupyter-widgets/base","base/js/dialog"], function(__WEBPACK_EXTERNAL_MODULE_139__,
# to
# define(["@jupyter-widgets/base"], function(__WEBPACK_EXTERNAL_MODULE_139__,
#
# matplotlib is troublesome on windows as well, but I have not found a solution...
#
# more qgrid woes
# qgrid problem on windows, see: https://github.com/quantopian/qgrid/issues/253
# jupyter nbextension enable --py --sys-prefix qgrid
# jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [5]:
# project folder structure
#
# example_project/
# ├── crystal_images
# ├── crystal_screen
# ├── database
# └── workflow
#     ├── 1-inspect
#     ├── 2-soak
#     └── 3-mount
#     └── 4-mount-manual

In [6]:
# Notes about the workflow
# 1-inspect
# User needs to save csv file after shifter inspection as _inspect.csv
# 2-soak
# Software writes compound csv plate
# In shifter:
# Load compound csv
# Load inspect csv
# Note: shifter software updates csv files on the fly (i.e. this may be relevant for the possible overwriting of the inspect csv file)
# Note2: destination information is stored in the compounds plate

In [7]:
lp3_project_folder = os.path.join('/Users/tobkro/tmp/jupyter_test')
db_template = os.path.join(os.getcwd(),'database', 'fragmax.sqlite')
shifter_template = os.path.join(os.getcwd(),'templates','shifter_template.csv')
crystal_plate_template = os.path.join(os.getcwd(),'templates','crystal_plate_template.csv')

In [8]:
logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.info('starting session')

In [9]:
settingsObject = settings()
dbObject = db_objects()
headerObject = page_header()
crystalplateObject = crystal_plate(settingsObject, dbObject, logger)
projectObject = project_description(settingsObject, dbObject, crystalplateObject, logger, lp3_project_folder, db_template)
crystalscreenObject = crystal_screen(settingsObject, dbObject, logger, crystal_plate_template)
inspectplateObject = inspect_plate(settingsObject, dbObject, logger)
soakplateObject = soak_plate(settingsObject, dbObject, logger)
crystalsoakObject = crystal_soak(settingsObject, dbObject, logger)
mountedcrystalsObject = mounted_crystals(settingsObject, dbObject, logger)

In [12]:
tab0 = VBox(children=[projectObject.grid_widget])
tab1 = VBox(children=[crystalscreenObject.grid_widget, HBox([crystalscreenObject.screen_sheet], layout={'height': '250px'}),
                      crystalscreenObject.save_screen_to_db_button, crystalscreenObject.crystal_screen_progress ])
tab2 = VBox(children=[crystalplateObject.grid_widget_upper, crystalplateObject.grid_widget_lower])
tab3 = VBox(children=[inspectplateObject.import_shifter_marked_crystals_button, 
                      inspectplateObject.inspected_wells_sheet])
tab4 = VBox(children=[soakplateObject.grid_widget])
tab5 = VBox(children=[crystalsoakObject.grid_widget])

tab = widgets.Tab(children=[tab0, tab1, tab2, tab3, tab4, tab5])
tab.set_title(0, 'Project Description')
tab.set_title(1, 'Crystal Screen')
tab.set_title(2, 'Crystal Plate')
tab.set_title(3, 'Inspect Plate')
tab.set_title(4, 'Soak Plate')
tab.set_title(5, 'Crystal Soak')
tab.set_title(6, 'Mounted Crystals')
VBox(children=[headerObject.logo, tab])


In [11]:
handler.show_logs()

Output(layout=Layout(border='1px solid black', height='160px', width='100%'), outputs=({'name': 'stdout', 'out…